In [7]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = web.DataReader('BTC-JPY', 'yahoo', start, end)
# data = web.DataReader('^N225', 'yahoo', start, end)
data = data.astype("double")
print(data)

initial = 10_000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

                  High         Low       Open       Close        Volume  \
Date                                                                      
2021-09-14  5181479.50  4921861.50  4945451.5  5166778.00  4.240741e+12   
2021-09-15  5296968.50  5123403.50  5167382.0  5269729.00  3.334521e+12   
2021-09-16  5303107.00  5166541.00  5267821.5  5241810.50  3.484527e+12   
2021-09-17  5290284.50  5149142.50  5240455.0  5196071.00  3.158009e+12   
2021-09-18  5363631.50  5176258.00  5196731.5  5307192.00  3.141290e+12   
...                ...         ...        ...         ...           ...   
2022-09-09  3055224.25  2779062.75  2780884.5  3046922.25  6.907153e+12   
2022-09-10  3100949.25  3016649.75  3046318.0  3089586.50  5.260394e+12   
2022-09-11  3103808.25  3050597.75  3089302.0  3103623.25  4.917772e+12   
2022-09-12  3198483.25  3088417.25  3103750.5  3189668.00  7.159440e+12   
2022-09-14  2953935.75  2909225.00  2911983.0  2909225.00  7.023019e+12   

             Adj Close  

**ボリンジャーバンドとRSI組み合わせ**

In [8]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.BBandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.BBandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   60.547945
Equity Final [$]               7787968.529838
Equity Peak [$]               10270668.983825
Return [%]                         -22.120315
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                  -22.120315
Volatility (Ann.) [%]               27.841553
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -34.117184
Avg. Drawdown [%]                  -34.117184
Max. Drawdown Duration      298 days 00:00:00
Avg. Drawdown Duration      298 days 00:00:00
# Trades                                   30
Win Rate [%]                        33.333333
Best Trade [%]                      29.669562
Worst Trade [%]                    -14.465488
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,66,74,6.534513e+06,6089680.00,-4.448330e+05,-0.068074,2021-11-19,2021-11-27,8 days
1,1,74,82,6.119824e+06,5550177.50,-5.696464e+05,-0.093082,2021-11-27,2021-12-05,8 days
2,1,82,83,5.577651e+06,5585922.00,8.271121e+03,0.001483,2021-12-05,2021-12-06,1 days
3,1,83,114,5.613572e+06,5059130.00,-5.544423e+05,-0.098768,2021-12-06,2022-01-06,31 days
4,1,114,115,5.084173e+06,5000048.50,-8.412419e+04,-0.016546,2022-01-06,2022-01-07,1 days
5,1,115,116,5.024799e+06,4801785.00,-2.230137e+05,-0.044383,2022-01-07,2022-01-08,1 days
6,1,116,117,4.825554e+06,4824033.50,-1.520336e+03,-0.000315,2022-01-08,2022-01-09,1 days
7,1,117,130,4.847912e+06,4146638.25,-7.012742e+05,-0.144655,2022-01-09,2022-01-22,13 days
8,1,130,131,4.167164e+06,3984710.50,-1.824536e+05,-0.043784,2022-01-22,2022-01-23,1 days
9,1,131,132,4.004435e+06,4127906.25,1.234714e+05,0.030834,2022-01-23,2022-01-24,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   46.849315
Equity Final [$]              10624940.382338
Equity Peak [$]               11951646.610925
Return [%]                           6.249404
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                    6.249404
Volatility (Ann.) [%]               41.083053
Sharpe Ratio                         0.152116
Sortino Ratio                        0.249752
Calmar Ratio                         0.231886
Max. Drawdown [%]                  -26.950284
Avg. Drawdown [%]                   -9.683202
Max. Drawdown Duration      132 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                                   39
Win Rate [%]                        41.025641
Best Trade [%]                      21.642109
Worst Trade [%]                    -13.558893
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,66,67,6.534513e+06,6624828.50,9.031548e+04,0.013821,2021-11-19,2021-11-20,1 days
1,1,74,77,6.119824e+06,6581385.00,4.615611e+05,0.075421,2021-11-27,2021-11-30,3 days
2,1,82,83,5.577651e+06,5585922.00,8.271121e+03,0.001483,2021-12-05,2021-12-06,1 days
3,1,83,87,5.613572e+06,5404596.00,-2.089763e+05,-0.037227,2021-12-06,2021-12-10,4 days
4,1,87,91,5.431349e+06,5305301.50,-1.260473e+05,-0.023207,2021-12-10,2021-12-14,4 days
5,1,91,114,5.331563e+06,5059130.00,-2.724327e+05,-0.051098,2021-12-14,2022-01-06,23 days
6,1,114,115,5.084173e+06,5000048.50,-8.412419e+04,-0.016546,2022-01-06,2022-01-07,1 days
7,1,115,116,5.024799e+06,4801785.00,-2.230137e+05,-0.044383,2022-01-07,2022-01-08,1 days
8,1,116,117,4.825554e+06,4824033.50,-1.520336e+03,-0.000315,2022-01-08,2022-01-09,1 days
9,1,117,122,4.847912e+06,4861291.00,1.337853e+04,0.002760,2022-01-09,2022-01-14,5 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,28,32,6.490644e+06,7046928.50,-5.562840e+05,-0.085706,2021-10-12,2021-10-16,4 days
1,-1,32,56,7.012046e+06,7650345.50,-6.382993e+05,-0.091029,2021-10-16,2021-11-09,24 days
2,-1,56,57,7.612476e+06,7560003.00,5.247329e+04,0.006893,2021-11-09,2021-11-10,1 days
3,-1,57,66,7.522581e+06,6502326.50,1.020254e+06,0.135626,2021-11-10,2021-11-19,9 days
4,1,66,74,6.534513e+06,6089680.00,-4.448330e+05,-0.068074,2021-11-19,2021-11-27,8 days
5,1,74,82,6.119824e+06,5550177.50,-5.696464e+05,-0.093082,2021-11-27,2021-12-05,8 days
6,1,82,83,5.577651e+06,5585922.00,8.271121e+03,0.001483,2021-12-05,2021-12-06,1 days
7,1,83,114,5.613572e+06,5059130.00,-5.544423e+05,-0.098768,2021-12-06,2022-01-06,31 days
8,1,114,115,5.084173e+06,5000048.50,-8.412419e+04,-0.016546,2022-01-06,2022-01-07,1 days
9,1,115,116,5.024799e+06,4801785.00,-2.230137e+05,-0.044383,2022-01-07,2022-01-08,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   91.232877
Equity Final [$]              14769052.996375
Equity Peak [$]               14778167.911775
Return [%]                           47.69053
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                    47.69053
Volatility (Ann.) [%]               80.333757
Sharpe Ratio                         0.593655
Sortino Ratio                         1.42053
Calmar Ratio                         1.499258
Max. Drawdown [%]                  -31.809419
Avg. Drawdown [%]                   -8.279911
Max. Drawdown Duration      191 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                   69
Win Rate [%]                        43.478261
Best Trade [%]                      26.842134
Worst Trade [%]                    -13.558893
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,32,56,7.012046e+06,7650345.50,-6.382993e+05,-0.091029,2021-10-16,2021-11-09,24 days
1,-1,56,57,7.612476e+06,7560003.00,5.247329e+04,0.006893,2021-11-09,2021-11-10,1 days
2,-1,57,66,7.522581e+06,6502326.50,1.020254e+06,0.135626,2021-11-10,2021-11-19,9 days
3,1,66,67,6.534513e+06,6624828.50,9.031548e+04,0.013821,2021-11-19,2021-11-20,1 days
4,-1,67,71,6.592036e+06,6628232.50,-3.619690e+04,-0.005491,2021-11-20,2021-11-24,4 days
...,...,...,...,...,...,...,...,...,...,...
64,4,347,348,2.801370e+06,2756945.00,-1.776999e+05,-0.015858,2022-08-27,2022-08-28,1 days
65,4,348,349,2.770592e+06,2710110.75,-2.419245e+05,-0.021830,2022-08-28,2022-08-29,1 days
66,4,349,361,2.723526e+06,3046318.00,1.291169e+06,0.118520,2022-08-29,2022-09-10,12 days
67,-4,361,364,3.031239e+06,2911983.00,4.770229e+05,0.039342,2022-09-10,2022-09-14,4 days


<Strategy BBandRSI_WithShortPosition(upper_bound=50,lower_bound=40,rsi_window=28)>

**MACDとRSIの組み合わせ**

In [9]:
import BacktestingStrategies.Strategy_MACDandRSI as btstMACDandRsi
importlib.reload(btstMACDandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandRsi.MACDandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    upper_bound = range(50, 85, 5),\
    lower_bound = range(10, 45, 5),\
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandRsi.MACDandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                   10000000.0
Equity Peak [$]                    10000000.0
Return [%]                                0.0
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 524 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    6.575342
Equity Final [$]                13072604.3625
Equity Peak [$]                 13072604.3625
Return [%]                          30.726044
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                   30.726044
Volatility (Ann.) [%]               24.165244
Sharpe Ratio                         1.271497
Sortino Ratio                        3.953226
Calmar Ratio                         2.917576
Max. Drawdown [%]                  -10.531361
Avg. Drawdown [%]                   -4.014935
Max. Drawdown Duration       10 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      17.062534
Worst Trade [%]                     13.340107
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,140,147,4.452199e+06,5046127.5,1.187856e+06,0.133401,2022-02-01,2022-02-08,7 days
1,4,294,309,2.761530e+06,3232717.0,1.884748e+06,0.170625,2022-07-05,2022-07-20,15 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,35,36,7.055731e+06,7363574.50,-3.078436e+05,-0.043630,2021-10-19,2021-10-20,1 days
1,-1,36,37,7.327125e+06,7549913.00,-2.227882e+05,-0.030406,2021-10-20,2021-10-21,1 days
2,-1,37,40,7.512541e+06,6970707.50,5.418334e+05,0.072124,2021-10-21,2021-10-24,3 days
3,-1,40,114,6.936202e+06,5059130.00,1.877072e+06,0.270620,2021-10-24,2022-01-06,74 days
4,-2,114,130,5.034087e+06,4146638.25,1.774898e+06,0.176288,2022-01-06,2022-01-22,16 days
5,-3,130,158,4.126112e+06,4602393.00,-1.428842e+06,-0.115431,2022-01-22,2022-02-19,28 days
6,-2,158,174,4.579611e+06,4417545.00,3.241323e+05,0.035389,2022-02-19,2022-03-07,16 days
7,-2,174,178,4.395678e+06,4582254.00,-3.731517e+05,-0.042445,2022-03-07,2022-03-11,4 days
8,-2,178,195,4.559572e+06,5724959.50,-2.330775e+06,-0.255591,2022-03-11,2022-03-28,17 days
9,-1,195,196,5.696621e+06,5822226.50,-1.256055e+05,-0.022049,2022-03-28,2022-03-29,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 535 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   84.657534
Equity Final [$]              22278702.486575
Equity Peak [$]               24101126.697525
Return [%]                         122.787025
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                  122.787025
Volatility (Ann.) [%]              116.448241
Sharpe Ratio                         1.054434
Sortino Ratio                        4.042122
Calmar Ratio                         4.360592
Max. Drawdown [%]                  -28.158339
Avg. Drawdown [%]                   -5.223014
Max. Drawdown Duration      142 days 00:00:00
Avg. Drawdown Duration       14 days 00:00:00
# Trades                                   27
Win Rate [%]                        40.740741
Best Trade [%]                      42.290831
Worst Trade [%]                    -21.529619
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,56,57,7.612476e+06,7560003.00,5.247329e+04,0.006893,2021-11-09,2021-11-10,1 days
1,-1,57,114,7.522581e+06,5059130.00,2.463451e+06,0.327474,2021-11-10,2022-01-06,57 days
2,-2,114,127,5.034087e+06,4854156.50,3.598616e+05,0.035742,2022-01-06,2022-01-19,13 days
3,2,127,128,4.878185e+06,4773439.00,-2.094911e+05,-0.021472,2022-01-19,2022-01-20,1 days
4,-2,128,147,4.749810e+06,5046127.50,-5.926340e+05,-0.062385,2022-01-20,2022-02-08,19 days
5,-2,147,148,5.021149e+06,5094823.00,-1.473477e+05,-0.014673,2022-02-08,2022-02-09,1 days
6,-2,148,149,5.069604e+06,5126716.50,-1.142257e+05,-0.011266,2022-02-09,2022-02-10,1 days
7,-2,149,155,5.101339e+06,5155098.50,-1.075185e+05,-0.010538,2022-02-10,2022-02-16,6 days
8,-2,155,161,5.129581e+06,4248544.50,1.762073e+06,0.171756,2022-02-16,2022-02-22,6 days
9,-3,161,175,4.227514e+06,4390929.50,-4.902459e+05,-0.038655,2022-02-22,2022-03-08,14 days


<Strategy MACDandRSI_WithShortPosition(MACDshort=20,MACDlong=40,MACDsignal=10,upper_bound=65,lower_bound=40,rsi_window=12)>

**MACDとボリンジャーバンドの組み合わせ**

In [10]:
import BacktestingStrategies.Strategy_MACDandBBD as btstMACDandBB
importlib.reload(btstMACDandBB)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandBB.MACDandBBD, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandBB.MACDandBBD_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   10.684932
Equity Final [$]                7748394.58625
Equity Peak [$]                10410061.23965
Return [%]                         -22.516054
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                  -22.516054
Volatility (Ann.) [%]               13.336195
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -28.191583
Avg. Drawdown [%]                  -28.191583
Max. Drawdown Duration      169 days 00:00:00
Avg. Drawdown Duration      169 days 00:00:00
# Trades                                    5
Win Rate [%]                             20.0
Best Trade [%]                       5.904799
Worst Trade [%]                     -27.56283
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,192,195,5.405760e+06,5724959.5,3.191993e+05,0.059048,2022-03-25,2022-03-28,3 days
1,1,195,204,5.753298e+06,5631946.5,-1.213515e+05,-0.021093,2022-03-28,2022-04-06,9 days
2,2,266,273,4.167187e+06,3018592.5,-2.297189e+06,-0.275628,2022-06-07,2022-06-14,7 days
3,2,308,324,3.121799e+06,3060081.0,-1.234351e+05,-0.019770,2022-07-19,2022-08-04,16 days
4,2,364,364,2.926397e+06,2911983.0,-2.882863e+04,-0.004926,2022-09-14,2022-09-14,0 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   15.068493
Equity Final [$]                7647365.08625
Equity Peak [$]                10410061.23965
Return [%]                         -23.526349
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                  -23.526349
Volatility (Ann.) [%]                14.52591
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -30.939352
Avg. Drawdown [%]                  -30.939352
Max. Drawdown Duration      169 days 00:00:00
Avg. Drawdown Duration      169 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                       5.904799
Worst Trade [%]                     -27.56283
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,192,195,5.405760e+06,5724959.5,3.191993e+05,0.059048,2022-03-25,2022-03-28,3 days
1,1,195,205,5.753298e+06,5345902.0,-4.073960e+05,-0.070811,2022-03-28,2022-04-07,10 days
2,2,266,273,4.167187e+06,3018592.5,-2.297189e+06,-0.275628,2022-06-07,2022-06-14,7 days
3,2,308,339,3.121799e+06,3152588.5,6.157992e+04,0.009863,2022-07-19,2022-08-19,31 days
4,2,364,364,2.926397e+06,2911983.0,-2.882863e+04,-0.004926,2022-09-14,2022-09-14,0 days


<Strategy MACDandBBD(MACDshort=15,MACDlong=30,MACDsignal=10)>

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   89.041096
Equity Final [$]               9485974.457375
Equity Peak [$]               14082012.032863
Return [%]                          -5.140255
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                   -5.140255
Volatility (Ann.) [%]               45.166763
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -37.81287
Avg. Drawdown [%]                   -6.801975
Max. Drawdown Duration      235 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   15
Win Rate [%]                             40.0
Best Trade [%]                      27.061962
Worst Trade [%]                     -27.56283
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,40,114,6.936202e+06,5059130.00,1.877072e+06,0.270620,2021-10-24,2022-01-06,74 days
1,-2,114,130,5.034087e+06,4146638.25,1.774898e+06,0.176288,2022-01-06,2022-01-22,16 days
2,-3,130,158,4.126112e+06,4602393.00,-1.428842e+06,-0.115431,2022-01-22,2022-02-19,28 days
3,-2,158,174,4.579611e+06,4417545.00,3.241323e+05,0.035389,2022-02-19,2022-03-07,16 days
4,-2,174,178,4.395678e+06,4582254.00,-3.731517e+05,-0.042445,2022-03-07,2022-03-11,4 days
5,-2,178,192,4.559572e+06,5379133.50,-1.639123e+06,-0.179745,2022-03-11,2022-03-25,14 days
6,1,192,195,5.405760e+06,5724959.50,3.191993e+05,0.059048,2022-03-25,2022-03-28,3 days
7,1,195,204,5.753298e+06,5631946.50,-1.213515e+05,-0.021093,2022-03-28,2022-04-06,9 days
8,-1,204,266,5.604068e+06,4146661.25,1.457407e+06,0.260062,2022-04-06,2022-06-07,62 days
9,2,266,273,4.167187e+06,3018592.50,-2.297189e+06,-0.275628,2022-06-07,2022-06-14,7 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   88.767123
Equity Final [$]                10099156.4269
Equity Peak [$]                14143316.44485
Return [%]                           0.991564
Buy & Hold Return [%]              -43.693633
Return (Ann.) [%]                    0.991564
Volatility (Ann.) [%]               53.077119
Sharpe Ratio                         0.018682
Sortino Ratio                        0.028972
Calmar Ratio                          0.02594
Max. Drawdown [%]                  -38.225296
Avg. Drawdown [%]                    -7.61968
Max. Drawdown Duration      110 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                   14
Win Rate [%]                             50.0
Best Trade [%]                      26.547547
Worst Trade [%]                     -27.56283
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,41,114,6.887626e+06,5059130.00,1.828496e+06,0.265475,2021-10-25,2022-01-06,73 days
1,-2,114,129,5.034087e+06,4639416.00,7.893426e+05,0.078400,2022-01-06,2022-01-21,15 days
2,-2,129,160,4.616451e+06,4417142.00,3.986178e+05,0.043174,2022-01-21,2022-02-21,31 days
3,-2,160,174,4.395277e+06,4417545.00,-4.453571e+04,-0.005066,2022-02-21,2022-03-07,14 days
4,-2,174,179,4.395678e+06,4550126.00,-3.088957e+05,-0.035136,2022-03-07,2022-03-12,5 days
5,-2,179,192,4.527603e+06,5379133.50,-1.703061e+06,-0.188075,2022-03-12,2022-03-25,13 days
6,2,192,195,5.405760e+06,5724959.50,6.383986e+05,0.059048,2022-03-25,2022-03-28,3 days
7,2,195,205,5.753298e+06,5345902.00,-8.147921e+05,-0.070811,2022-03-28,2022-04-07,10 days
8,-2,205,266,5.319440e+06,4146661.25,2.345557e+06,0.220470,2022-04-07,2022-06-07,61 days
9,3,266,273,4.167187e+06,3018592.50,-3.445784e+06,-0.275628,2022-06-07,2022-06-14,7 days


<Strategy MACDandBBD_WithShortPosition(MACDshort=15,MACDlong=30,MACDsignal=10)>